<a href="https://colab.research.google.com/github/Evolverive/CountSemiprimes/blob/master/ultralytics_YOLOv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://storage.googleapis.com/ultralytics/logo/logoname1000.png" width="150">

This notebook contains software developed by Ultralytics LLC, and **is freely available for redistribution under the GPL-3.0 license**. For more information please visit https://github.com/ultralytics/yolov3 and www.ultralytics.com.





In [0]:
# Import
import time
import glob
import torch
import os

from IPython.display import clear_output
from google.colab import files
from google.colab import drive
print(torch.__version__, torch.cuda.get_device_properties(0))

In [0]:
# Install
t = time.time()
!rm -rf yolov3 weights coco cocoapi apex
!git clone https://github.com/ultralytics/yolov3
!git clone https://github.com/cocodataset/cocoapi && cd cocoapi/PythonAPI && make && cd ../.. && cp -r cocoapi/PythonAPI/pycocotools yolov3
!git clone https://github.com/NVIDIA/apex && cd apex && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" . --user && cd .. && rm -rf apex
!bash yolov3/data/get_coco_dataset_gdrive.sh  # copy COCO2014 dataset (20GB)
%cd yolov3
clear_output(); print('Done (%.2fs)' % (time.time() - t))

In [0]:
# Detect
!python3 detect.py  # detect 2 persons, 1 tie

In [0]:
# Train
!python3 train.py --data data/coco_64img.data --img-size 320 --single-scale --epochs 10 --batch-size 32 --accumulate 2 --giou

In [0]:
%cd ..  # change directory

In [0]:
!ls  # see files

In [0]:
# Reproduce mAP
!python3 test.py --save-json --img-size 608
!python3 test.py --save-json --img-size 416
!python3 test.py --save-json --img-size 320

In [0]:
# Unit Tests
!python3 detect.py  # detect 2 persons, 1 tie
!python3 test.py --data data/coco_32img.data  # test mAP = 0.8
!python3 train.py --data data/coco_32img.data --epochs 4 --nosave  # train 4 epochs

In [0]:
# Reproduce Tutorials
!python3 train.py --nosave --data data/coco_64img.data && mv results.txt results4_64img.txt
!python3 -c "from utils import utils; utils.plot_results()"
[files.download(f) for f in glob.glob('results*.*')]

In [0]:
# Evolve Hyperparameters
!python3 train.py --data data/coco_100img.data --evolve

In [0]:
# Plot Results
!python3 -c "from utils import utils; utils.plot_results()"
[files.download(f) for f in glob.glob('results*.*')]

In [0]:
# Darknet ----------------------------------------------------------------------
# Copy dataset
!rm -rf supermarket2 && cp -r 'gdrive/My Drive/cloud/data/supermarket2' .  # from drive
# !gsutil cp -r gs://sm6/supermarket2 .  # from bucket

# Install Darknet
!rm -rf darknet && git clone https://github.com/AlexeyAB/darknet 
%cd darknet 
!wget -c https://pjreddie.com/media/files/darknet53.conv.74  # sudo apt install libopencv-dev && make
!cp -r '../gdrive/My Drive/cloud/darknet/cfg/.' ../ # from drive
!cp '../gdrive/My Drive/cloud/darknet/Makefile' . && make
clear_output();

# Train
!./darknet detector train ../supermarket2/supermarket2.data ../yolov3-spp-sm2-1cls-scalexy.cfg darknet53.conv.74 -map -dont_show # train spp
!cp -r backup/*5000.weights '../gdrive/My Drive/cloud/darknet'

In [0]:
!gsutil cp yolov3/train.py gs://yolov4